In [1]:
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from scipy.stats import pearsonr
import tensorflow as tf
import torch
import torchvision
sys.path.append('/home/juan/Documents/Master_Project/code/my_work/Testing-Algorithms/')
from rf_improved2_copy import TorchRandomForestRegressor
from tqdm import tqdm
from math import sqrt
import warnings
from sklearn.exceptions import DataConversionWarning


2024-03-27 13:58:51.013191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 13:58:51.123323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 13:58:51.986216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:


data1 =pd.read_csv('data1.csv')
data9 = pd.read_csv('data_1_test.csv')

df_1 = data1.iloc[:, 1:]
df_9 = data9.iloc[:,1:]

columns = df_1.columns.tolist()

columns.remove("Folder")
columns.remove("QM/MM SP Barrier")

new_columns_order = columns + ["Folder", "QM/MM SP Barrier"]

df_1 = df_1.reindex(columns=new_columns_order)
df_1.head()

,661,666,156,444,705,407,807,650,356,699,...,Box_26,Box_33,Box_34,Box_46,Box_50,Box_51,Total Charge,internal_vdw_energy,Folder,QM/MM SP Barrier
0,0,0,1,1,0,0,1,1,2,0,...,-0.241552,0.0,0.0,0.000000,-0.758976,-1.007604,1.000009,2.925351,mol_134810,29.71
1,0,0,0,0,0,1,0,0,5,0,...,0.000000,0.0,0.0,-0.056827,-0.020914,-0.183128,-0.000017,-4.432160,mol_147462,28.33
2,0,0,0,0,0,0,1,2,4,0,...,-1.176918,0.0,0.0,-0.249862,-0.349354,0.000000,0.000008,-5.104365,mol_22419,28.28
3,0,0,0,0,0,1,1,1,2,0,...,0.096336,0.0,0.0,0.000000,-0.883456,0.819872,-0.000009,-3.150597,mol_469,22.72
4,1,0,0,0,0,0,1,1,3,1,...,0.026424,0.0,0.0,0.000000,0.662524,-1.430314,0.999997,-2.510429,mol_67286,25.81


In [3]:
df = pd.read_csv('/home/juan/Documents/Master_Project/data/process/final_selection/All_feats_newC_hp_atoms.csv')
df = df.iloc[:,1:]
df.head()

,nBondsD,nBonds2,nBondsT,nBondsM,pubchemcomplexity,pubchemhbondacceptor,pubchemhbonddonor,pubchemrotbon,pubchemlogp,pubchemmass,...,hp_Atom_31,hp_Atom_32,hp_Atom_33,hp_Atom_34,hp_Atom_35,hp_Atom_36,hp_Atom_37,hp_Atom_38,Folder,QM/MM SP Barrier
0,1,11,0,11,0.39820,5,2,4,0.39820,285.371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_134810,29.71
1,0,14,0,12,3.35790,4,0,3,3.35790,309.413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_147462,28.33
2,2,14,0,11,1.79150,6,1,7,1.79150,340.427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_22419,28.28
3,1,16,1,6,1.46888,6,0,6,1.46888,316.405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_469,22.72
4,1,12,0,6,-0.04998,4,3,2,-0.04998,252.313,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_67286,25.81


In [4]:
import pandas as pd

dataset_initial = pd.read_csv('dataset.csv')

df00 = dataset_initial.iloc[:, 1:]

df00.drop(columns='total_charge', inplace=True)

df00.tail()


,0,1,2,3,4,5,6,7,8,9,...,resid 120,resid 119,resid 146,resid 145,resid 147,resid 786,resid 61,resid 29,Folder,QM/MM SP Barrier
1092,0,0,0,0,0,0,0,0,0,0,...,19.635709,22.622308,19.836394,20.933459,22.265739,15.378876,5.409570,20.505763,mol_178096,21.45
1093,0,0,0,0,0,0,0,0,0,0,...,18.754397,22.329591,19.444639,20.593992,21.991518,15.392180,4.830849,20.268452,mol_178159,27.00
1094,0,0,0,0,0,0,0,0,0,0,...,17.451327,21.067321,18.741670,19.913888,20.867617,13.842437,3.824708,18.920998,mol_178160,29.97
1095,0,0,0,0,0,0,0,0,0,0,...,19.142024,22.027824,19.150536,20.300864,21.613279,15.685708,5.110045,19.831135,mol_178411,28.04
1096,0,0,0,0,0,0,0,0,0,0,...,19.653265,22.952895,20.551297,21.449449,22.798516,16.306594,5.876727,21.242883,mol_178649,25.78


## Common functions

In [5]:
def preprocess_data_manual_no_split(df, use_torch):
    X = df.drop(columns=['QM/MM SP Barrier', 'Folder']).values
    Y = df['QM/MM SP Barrier'].values
    ss = StandardScaler()
    Y_reshaped = Y.reshape(-1, 1)
    Y_normalized = ss.fit_transform(Y_reshaped)
    if use_torch:
        X = torch.tensor(X, dtype=torch.float32)
        Y_normalized = torch.tensor(Y_normalized, dtype=torch.float32).flatten()
    return X, Y_normalized, ss

def mean_deviation_from_true_mean(y_true, y_pred):
    """
    Calculates the mean of the absolute deviations of predicted values from the mean of true values.

    :param y_true: array-like, true values.
    :param y_pred: array-like, predicted values.
    :return: float, mean deviation from the true mean.
    """
    true_mean = np.mean(y_true)
    deviations = np.abs(y_pred - true_mean)
    mean_deviation = np.mean(deviations)
    return mean_deviation

# Deep Neural Network

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Add
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


def dnn(num_features, use_residual_blocks=True):
    # Input Layer
    input_layer = Input(shape=(num_features,))

    # Initial Dense Block
    x = Dense(256, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # Residual Blocks
    if use_residual_blocks:
        for _ in range(5):
            block_input = x
            x = Dense(256, activation='relu')(x)
            x = BatchNormalization()(x)
            x = Dropout(0.3)(x)
            x = Dense(256, activation='relu')(x)
            x = BatchNormalization()(x)
            x = Dropout(0.3)(x)
            # Adding skip connection
            x = Add()([x, block_input])

    # Dense Layers
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Output Layer
    output = Dense(1)(x)  # Regression output

    # Construct and compile model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error',
                  metrics=[tf.keras.metrics.MeanSquaredError(), root_mean_squared_error])

    return model

#RMSE
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# Cross-validation function for DNN
def cross_validate_with_dnn(X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores, mae_scores, variance_scores, r2_scores, pearson_corr_scores, deviations = [], [], [], [], [], []


    for train_index, test_index in tqdm(kf.split(X), total=n_splits, desc="Cross-Validation"):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Initialize the DNN model
        model = dnn(num_features=X_train.shape[1])
        
        # Fit the model
        model.fit(X_train, y_train, epochs=40, batch_size=16, verbose=0) # Training
        
        # Predict
        y_pred = model.predict(X_test).flatten()
        y_test_flat = y_test.flatten()
        # Calculate metrics
        rmse_scores.append(np.sqrt(mean_squared_error(y_test_flat, y_pred)))
        mae_scores.append(mean_absolute_error(y_test_flat, y_pred))
        variance_scores.append(np.var(y_test_flat - y_pred))
        r2_scores.append(r2_score(y_test_flat, y_pred))
        pearson_corr_scores.append(pearsonr(y_test_flat, y_pred)[0])
        
        deviations.append(mean_deviation_from_true_mean(y_test_flat, y_pred))

    # Aggregate results
    mean_rmse = np.mean(rmse_scores)
    mean_mae = np.mean(mae_scores)
    mean_variance = np.mean(variance_scores)
    mean_r2 = np.mean(r2_scores)
    mean_pearson_corr = np.mean(pearson_corr_scores)
    mean_deviation = np.mean(deviations)

    return mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae

def complete_analysis_pipeline(df, title, use_torch=False):
    X, Y_normalized, ss = preprocess_data_manual_no_split(df, use_torch)
    train_percentage = 0.99
    n_samples = len(X)
    n_train_samples = int(train_percentage * n_samples)
    X_train, Y_train = X[:n_train_samples], Y_normalized[:n_train_samples]
    X_blind, Y_blind = X[n_train_samples:], Y_normalized[n_train_samples:]

    mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae = cross_validate_with_dnn(X_train, Y_train)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)



In [22]:
complete_analysis_pipeline(df_1, ' ')

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

2024-03-27 14:45:59.168650: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


Cross-Validation:  10%|█         | 1/10 [06:42<1:00:24, 402.76s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


Cross-Validation:  90%|█████████ | 9/10 [52:57<05:52, 352.15s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


Cross-Validation: 100%|██████████| 10/10 [58:57<00:00, 353.73s/it]

0.9563688941344781 0.3805575625264424 0.32036537 0.08486346028655395 0.8757564633103696 0.7003751832152034


In [23]:
complete_analysis_pipeline(df00, ' ')

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


Cross-Validation:  10%|█         | 1/10 [00:44<06:39, 44.37s/it]

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/stepWARNING:tensorflow:6 out of the last 35 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7896f67784a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


Cross-Validation:  20%|██        | 2/10 [01:33<06:17, 47.14s/it]

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


Cross-Validation:  90%|█████████ | 9/10 [06:14<00:42, 42.63s/it]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


Cross-Validation: 100%|██████████| 10/10 [07:09<00:00, 42.93s/it]

1.0449843902972646 0.11952676937662668 0.34569734 -0.10455492950764826 1.058745660348354 0.8207892535083194


In [24]:
complete_analysis_pipeline(df_9, ' ')

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


Cross-Validation:  10%|█         | 1/10 [05:30<49:34, 330.45s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


Cross-Validation:  20%|██        | 2/10 [10:58<43:52, 329.06s/it]

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


Cross-Validation:  90%|█████████ | 9/10 [50:59<05:49, 349.53s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


Cross-Validation: 100%|██████████| 10/10 [56:30<00:00, 339.05s/it]

1.0356320418401512 0.3767049435146374 0.3250959 -0.19619825098019655 1.192114379976031 0.681505276273945


In [25]:
complete_analysis_pipeline(df, ' ')

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


Cross-Validation:  10%|█         | 1/10 [04:56<44:26, 296.25s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


Cross-Validation:  20%|██        | 2/10 [10:28<42:21, 317.63s/it]

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


Cross-Validation:  90%|█████████ | 9/10 [49:07<05:26, 326.69s/it]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


Cross-Validation: 100%|██████████| 10/10 [54:11<00:00, 325.18s/it]

0.9341481487432233 0.39065902146582093 0.3024761 0.12685495791267504 0.8616286096412369 0.6881723213842322


# TAbNEt

In [28]:
from pytorch_tabnet.tab_model import TabNetRegressor
def cross_validate_with_tabnet(X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores, mae_scores, variance_scores, r2_scores, pearson_corr_scores, deviations = [], [], [], [], [], []

    # Standardize features
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)

    # Standardize target
    scaler_Y = StandardScaler()
    y_scaled = scaler_Y.fit_transform(y.reshape(-1, 1))  # Keep y_scaled as 2D

    # TabNet parameters
    tabnet_params = dict(n_d=64, n_a=64, n_steps=13, optimizer_fn=torch.optim.Adam,
                         optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                         mask_type='entmax',
                         scheduler_params=dict(mode="min", patience=5, min_lr=1e-5, factor=0.9,),
                         scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                         verbose=0)

    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y_scaled[train_index], y_scaled[test_index]

        clf = TabNetRegressor(**tabnet_params)
        clf.fit(X_train, y_train, max_epochs=40)

        preds = clf.predict(X_test)
        preds_unscaled = scaler_Y.inverse_transform(preds).flatten()
        y_test_unscaled = scaler_Y.inverse_transform(y_test).flatten()
        rmse_scores.append(np.sqrt(mean_squared_error(y_test_unscaled, preds_unscaled)))
        mae_scores.append(mean_absolute_error(y_test_unscaled, preds_unscaled))
        variance_scores.append(np.var(y_test_unscaled - preds_unscaled))
        r2_scores.append(r2_score(y_test_unscaled, preds_unscaled))
        pearson_corr_scores.append(pearsonr(y_test_unscaled, preds_unscaled)[0])
        deviations.append(mean_deviation_from_true_mean(y_test_unscaled, preds_unscaled))
        
    # Collect scores
    mean_rmse = np.mean(rmse_scores)
    mean_mae = np.mean(mae_scores)
    mean_variance = np.mean(variance_scores)
    mean_r2 = np.mean(r2_scores)
    mean_pearson_corr = np.mean(pearson_corr_scores)
    mean_deviation = np.mean(deviations)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)
    return mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae

def complete_analysis_pipeline(df, title):
    X, Y_normalized, ss = preprocess_data_manual_no_split(df, use_torch='False')
    train_percentage = 0.99
    n_samples = len(X)
    n_train_samples = int(train_percentage * n_samples)
    X_train, Y_train = X[:n_train_samples], Y_normalized[:n_train_samples]
    X_blind, Y_blind = X[n_train_samples:], Y_normalized[n_train_samples:]

    mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae = cross_validate_with_tabnet(X_train, Y_train)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)


In [30]:
complete_analysis_pipeline(df_1, ' ')

/home/juan/myenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


/home/juan/myenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/juan/myenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/juan/myenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/home/juan/myenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [ ]:
complete_analysis_pipeline(df00, ' ')

In [ ]:
complete_analysis_pipeline(df_9, ' ')

In [ ]:
complete_analysis_pipeline(df, ' ')